## Closer look at 5 loci between replicates

Dan suggested that I look at a small number of loci between replicate scallops to see how the genotyping error can be so high between the SNP and haplotype level. 


They look terribly alike. Which means the error could still be in my code. 

## Popmap with sample not included!

I just realized that I excluded FG100_A, but it's still in the popmap, and it got written into the tags GP file, although it's not in the haplotypes file, so that could be the problem. 

That would be embarrassing >.<

Checking here:

In [13]:
cd /mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts

/mnt/hgfs/SHARED_FOLDER/Git_repo/CRAGIG_run1/Scripts


In [14]:
!python parse_hapfile_to_tagGenePop.py \
-f ../../../WorkingFolder/Stacks_2/batch_100.haplotypes.tsv \
-p ../../../WorkingFolder/popmap_cragigrun1_fortagsGP.txt \
-o ../../../WorkingFolder/Stacks_2/tagsGenepop_20170324.genepop

This script found 24243 genotypes to have more than two alleles, and replaced them with missing values, 0000


Now re-check haplotype genotyping error:

In [15]:
cd /mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2

/mnt/hgfs/SHARED_FOLDER/WorkingFolder/Stacks_2


In [16]:
input = open("cragigrun1_reps_tag_gens.txt") # read in file
lines = input.readlines()
biglist = []
for line in lines:
    linelist = line.strip().split()
    biglist.append(linelist) # prepare a list to make an array

# check list of lists is what you expect
print biglist[0][0:9]

['FG101_A,', '0000', '0101', '0505', '0709', '0606', '0303', '0707', '0202']


In [17]:
headerline = lines[8]
tag_list = headerline.strip().split(",")

print tag_list[0:19]

['5', '6', '13', '16', '20', '22', '27', '28', '31', '35', '49', '50', '57', '62', '63', '71', '75', '77', '79']


In [18]:
num_tags = len(tag_list) # iterate over number of tags
print "You have " + str(num_tags) + " RAD tags."

You have 7749 RAD tags.


In [19]:
import numpy as np
myarray = np.array(biglist) # turn the file into an array

In [20]:
FG101 = np.array(biglist[0:2])
FG102 = np.array(biglist[2:4])
FG103 = np.array(biglist[4:6])
FG104 = np.array(biglist[6:8])

In [21]:
FG101[0:2,0:7]

array([['FG101_A,', '0000', '0101', '0505', '0709', '0606', '0303'],
       ['FG101_B,', '0000', '0101', '0505', '0709', '0606', '0303']], 
      dtype='|S8')

In [22]:
# put arrays in list
list_of_arrays = []
list_of_arrays.append(FG101)
list_of_arrays.append(FG102)
list_of_arrays.append(FG103)
list_of_arrays.append(FG104)

In [23]:
# initiate counts and lists to track genotyping error
match_genotypes_count = 0
match_genotypes_tags = []
partmismatch_genotypes_count = 0
partmismatch_genotypes_tags = []
totalmismatch_genotypes_count = 0
totalmismatch_genotypes_tags = []
nodata_mismatch_count = 0
nodata_mismatch_tags = []

In [24]:
# sort genotype match/mismatch by sample
bysam_match_tags = []
bysam_partmismatch_tags = []
bysam_totalmismatch_tags = []
bysam_missing_tags = []

timescount = 1
for array in list_of_arrays:
    sam_match_genotypes_count = 0
    sam_match_genotypes_tags = []
    sam_partmismatch_genotypes_count = 0
    sam_partmismatch_genotypes_tags = []
    sam_totalmismatch_genotypes_count = 0
    sam_totalmismatch_genotypes_tags = []
    sam_nodata_mismatch_count = 0
    sam_nodata_mismatch_tags = []
    for i in range(1,num_tags+1):
        A_gen = array[0][i] # get genotypes
        B_gen = array[1][i]
        A_1 = A_gen[0:2] # pull out alleles
        A_2 = A_gen[2:4]
        B_1 = B_gen[0:2]
        B_2 = B_gen[2:4]
        A_list = [] # store alleles in lists
        A_list.append(A_1)
        A_list.append(A_2)
        B_list = []
        B_list.append(B_1)
        B_list.append(B_2)
        first_test_string = B_1 + B_2
        second_test_string = B_2 + B_1
        match_options = []
        match_options.append(first_test_string)
        match_options.append(second_test_string)
        if (A_gen == "0000" or B_gen == "0000") or (A_gen == "0000" and B_gen == "0000"):
            nodata_mismatch_count += 1
            nodata_mismatch_tags.append(tag_list[i-1])
            sam_nodata_mismatch_count +=1
            sam_nodata_mismatch_tags.append(tag_list[i-1])
        elif A_gen in match_options: # if perfect match
            match_genotypes_count += 1
            match_genotypes_tags.append(tag_list[i-1])
            sam_match_genotypes_count += 1
            sam_match_genotypes_tags.append(tag_list[i-1])
        elif A_1 in B_list or A_2 in B_list: # if one allele is a match
            partmismatch_genotypes_count += 1
            partmismatch_genotypes_tags.append(tag_list[i-1])
            sam_partmismatch_genotypes_count += 1
            sam_partmismatch_genotypes_tags.append(tag_list[i-1])
        elif A_1 not in B_list or A_2 not in B_list: # if not a match at all
            totalmismatch_genotypes_count += 1
            totalmismatch_genotypes_tags.append(tag_list[i-1])
            sam_totalmismatch_genotypes_count += 1
            sam_totalmismatch_genotypes_tags.append(tag_list[i-1])
        else:
            print "Your loop may not be fully working." 
            print "You thought there were three possible options, but there were more than three. Check!"
        
    bysam_missing_tags.append(sam_nodata_mismatch_tags)
    bysam_match_tags.append(sam_match_genotypes_tags)
    bysam_partmismatch_tags.append(sam_partmismatch_genotypes_tags)
    bysam_totalmismatch_tags.append(sam_totalmismatch_genotypes_tags)

    print "\nSample number " + str(timescount) + " :"
    print "Genotypes with perfect match: " + str(sam_match_genotypes_count) + " : "
    print "Genotypes with partial match: " + str(sam_partmismatch_genotypes_count) + " : "
    print "Genotypes with total mismatch: " + str(sam_totalmismatch_genotypes_count) + " : "
    print "Genotypes with mismatch due to missing data: " + str(sam_nodata_mismatch_count) + " : "
        
    timescount +=1


Sample number 1 :
Genotypes with perfect match: 6891 : 
Genotypes with partial match: 104 : 
Genotypes with total mismatch: 2 : 
Genotypes with mismatch due to missing data: 752 : 

Sample number 2 :
Genotypes with perfect match: 7020 : 
Genotypes with partial match: 95 : 
Genotypes with total mismatch: 1 : 
Genotypes with mismatch due to missing data: 633 : 

Sample number 3 :
Genotypes with perfect match: 6939 : 
Genotypes with partial match: 151 : 
Genotypes with total mismatch: 3 : 
Genotypes with mismatch due to missing data: 656 : 

Sample number 4 :
Genotypes with perfect match: 6945 : 
Genotypes with partial match: 142 : 
Genotypes with total mismatch: 0 : 
Genotypes with mismatch due to missing data: 662 : 


In [25]:
from __future__ import division

total_excl_mv = match_genotypes_count + partmismatch_genotypes_count + totalmismatch_genotypes_count

print "\nTotal genotypes with perfect match: " + str(match_genotypes_count) + " : " + str((float(match_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Combined (both partial and total) genotyping error: " + str(float(partmismatch_genotypes_count+totalmismatch_genotypes_count)/float(total_excl_mv)*100)[0:5] + "%"
print "\nTotal genotypes with partial match: " + str(partmismatch_genotypes_count) + " : " + str((float(partmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with total mismatch: " + str(totalmismatch_genotypes_count) + " : " + str((float(totalmismatch_genotypes_count)/float(total_excl_mv)*100))[0:5] + "%"
print "Total genotypes with mismatch due to missing data: " + str(nodata_mismatch_count)
print "Total genotypes excluding those with missing data : " + str(total_excl_mv)


Total genotypes with perfect match: 27795 : 98.23%
Combined (both partial and total) genotyping error: 1.760%

Total genotypes with partial match: 492 : 1.738%
Total genotypes with total mismatch: 6 : 0.021%
Total genotypes with mismatch due to missing data: 2703
Total genotypes excluding those with missing data : 28293


# HALLELUJAH

![img](https://m.popkey.co/b44edc/mzxo3_s-200x150.gif)